In [ ]:
%pip install selenium
%pip install pandas

In [250]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import re
import pandas as pd
from bs4 import BeautifulSoup
import warnings

warnings.filterwarnings("ignore")

In [251]:

# Set up Chrome WebDriver
options = Options()
options.add_argument("--headless")  # Run Chrome in headless mode (without opening a browser window)
driver = webdriver.Chrome(options=options)

# Navigate to the page
url = "http://ned.ipac.caltech.edu/forms/OBJatt.html"
driver.get(url)
element = driver.find_element(By.XPATH,'//a[@href="javascript:unhide(\'selectOUT\');"]')
# Click the element using JavaScript
driver.execute_script('arguments[0].click();', element)
# # Find the checkbox element by its attributes
# checkbox = driver.find_element(By.NAME,'NO_LINKS')

# checkbox.click()
# Find the checkbox element by its attributes
checkbox = driver.find_element(By.XPATH,"//input[@type='checkbox' and @name='M' and @value='1168']")
checkbox.click()

submit_button = driver.find_element(By.XPATH, "//input[@type='submit'][@value='Submit Query']")
submit_button.click()


In [252]:
# Get the page source
page_source = driver.page_source

# Use regular expressions to find the text
page_info_pattern = r'Page \d+ of \d+'
page_info_match = re.search(page_info_pattern, page_source)

if page_info_match:
    page_info_text = page_info_match.group(0)
    page_number, total_pages = [int(part) for part in re.findall(r'\d+', page_info_text)]
    print("Page Number:", page_number)
    print("Total Pages:", total_pages)
else:
    # Handle case when the text is not found
    page_info_text = "Page information not found"

Page Number: 1
Total Pages: 381


In [253]:
current_page = 1  # Start from page 1
final_df = pd.DataFrame()
while True:

    # Locate the first table on the page using XPath
    table = driver.find_element(By.XPATH, '//table')
    # Extract the HTML content of the table
    table_html = table.get_attribute("outerHTML")
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(table_html, 'html.parser')

    # Find the <pre><strong> element that contains your data
    pre_strong = soup.find('pre')

    # Extract the text data from the <pre><strong> element
    data = pre_strong.get_text()

    # Split the data into lines
    lines = data.strip().split('\n')

    # Initialize an empty list to hold the split lines
    table_data = []

    for line in lines:
        # Skip lines that consist only of pipe characters
        if not re.match(r'^\|+$', line):
            # Use regular expression to split on '|' only if it's not within cell values
            row = re.split(r'(?<!\|)\|', line)

            # Clean up the row by removing leading and trailing spaces
            row = [cell.strip() for cell in row]

            # Append the cleaned row to the table data
            table_data.append(row)

    # Create a DataFrame from the table data
    df = pd.DataFrame(table_data)

    # Clean up the DataFrame by removing any leading or trailing spaces
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    # Delete the first four rows (index 0 to 3)
    df = df.drop(index=range(4))

    # Delete the first and last columns
    df = df.drop(df.columns[[0,1, -1]], axis=1)

    # Reset the index to start from 0
    df = df.reset_index(drop=True)

    # Append the table data to the final_df
    final_df = pd.concat([final_df, df], ignore_index=True)
    print('size of df',final_df.shape)

    if current_page == total_pages:
        break  # Break out of the loop if you're on the last page

    links = driver.find_elements(By.XPATH, '//a[contains(@href, "/cgi-bin/OBJatt?")]')

    # Access the last link in the list and click it
    if links:
        last_link = links[-1]
        last_link.click()

    current_page += 1  # Increment the current page number

size of df (100, 8)
size of df (200, 8)
size of df (300, 8)
size of df (400, 8)
size of df (500, 8)
size of df (600, 8)
size of df (700, 8)
size of df (800, 8)
size of df (900, 8)
size of df (1000, 8)
size of df (1100, 8)
size of df (1200, 8)
size of df (1300, 8)
size of df (1400, 8)
size of df (1500, 8)
size of df (1600, 8)
size of df (1700, 8)
size of df (1800, 8)
size of df (1900, 8)
size of df (2000, 8)
size of df (2100, 8)
size of df (2200, 8)
size of df (2300, 8)
size of df (2400, 8)
size of df (2500, 8)
size of df (2600, 8)
size of df (2700, 8)
size of df (2800, 8)
size of df (2900, 8)
size of df (3000, 8)
size of df (3100, 8)
size of df (3200, 8)
size of df (3300, 8)
size of df (3400, 8)
size of df (3500, 8)
size of df (3600, 8)
size of df (3700, 8)
size of df (3800, 8)
size of df (3900, 8)
size of df (4000, 8)
size of df (4100, 8)
size of df (4200, 8)
size of df (4300, 8)
size of df (4400, 8)
size of df (4500, 8)
size of df (4600, 8)
size of df (4700, 8)
size of df (4800, 8)
s

In [255]:
final_df.columns = ["Object Name", "RA", "Dec", "Object Type", "Redshift", "z Qual", "Galaxy Morphology","Activity Type","extra"]

In [256]:
driver.quit()

In [257]:
final_df.to_csv('Galaxy_Morphology_S_Catalogue.csv')

In [263]:
df= pd.read_csv('Galaxy_Morphology_S0_Catalogue.csv')

In [265]:
df.shape

(12551, 8)

In [259]:
df.head()

,Object Name,RA,Dec,Object Type,Redshift,z Qual,Galaxy Morphology,Activity Type
0,DUKST 537-038,00h00m02.72s,-23d22m03.8s,G,0.065806,SUN,S0,NaN
1,WISEA J000011.94-663046.2,00h00m11.93s,-66d30m46.0s,G,0.072564,SUN,S0,NaN
2,FAIRALL 1061,00h00m23.61s,-47d01m07.7s,G,0.019984,SLS,S0,NaN
3,NGC 7807,00h00m26.58s,-18d50m30.8s,G,0.025534,SLS,Sb?,NaN
4,ESO 193-IG 008,00h00m32.30s,-50d08m54.0s,GPair,NaN,NaN,pair,NaN


In [260]:
df['Dec'].dtype

dtype('O')

In [261]:
filtered_df = df[~df['Dec'].str.contains('\+')]

In [262]:
filtered_df.shape

(7600, 8)

In [266]:
# Convert the 'column_name' to a float
filtered_df['Redshift'] = filtered_df['Redshift'].astype(float)

# Filter rows where 'column_name' is less than or equal to 0.26
filtered_df = filtered_df[filtered_df['Redshift'] <= 0.26]

In [267]:
filtered_df.shape

(7345, 8)